In [5]:
!pip install pandas
!pip install xlrd
!pip install 'pandas-profiling==2.8.0'

     |████████████████████████████████| 103 kB 174 kB/s 


In [83]:
import pandas as pd
from pandas_profiling import ProfileReport

In [133]:
df_hurtos = pd.concat([
    pd.read_excel('data/hurto-de-automotores-2017_0.xls', skiprows=9),
    pd.read_excel('data/hurto-de-automotores-2018_0.xls', skiprows=8),
    pd.read_excel('data/hurto-de-automotores-2018_1.xls', skiprows=9),
    pd.read_excel('data/hurto-de-automotores-2019_0.xls', skiprows=8)
])

# Limpieza

In [134]:
# Correción de valores de edad y cantidad. 
# Edad -1 representa que no se conoce la edad del individuo
# Cantidad 1000 se borra en un paso posterior.
df_hurtos['Edad'] = df_hurtos['Edad'].fillna('-1')
df_hurtos['Edad'] = df_hurtos['Edad'].astype(str)
df_hurtos = df_hurtos[df_hurtos['Edad'] != '-']
df_hurtos['Cantidad'] = df_hurtos['Cantidad'].fillna(1000)

# Corrección de tipos.
cols_str = [
    'Fecha','Hora', 'Departamento', 'Municipio', 'Día', 'Barrio', 'Zona', 'Clase de sitio', 
    'Arma empleada', 'Móvil Agresor', 'Móvil Victima', 'Sexo', 'Estado civil' ,
    'País de nacimiento','Clase de empleado', 'Profesión', 'Escolaridad', 'Código DANE', 'CLASE',
    'MARCA', 'LINEA', 'MODELO', 'COLOR']
cols_int = ['Edad', 'Cantidad']
for col in cols_str:
    df_hurtos[col] = df_hurtos[col].astype(str)

for col in cols_int:
    df_hurtos[col] = df_hurtos[col].astype('int64')

# Remoción de duplicados
df_hurtos.drop_duplicates(inplace=True)

# Remoción de valores atípicos. 
# 1. Cantidad de vehículos robados con valores mayores 61 y >= 7905, que representan menos del 1% de los datos
df_hurtos = df_hurtos[df_hurtos['Cantidad'] < 60]
# 2. Registros con edad mayor a 110
df_hurtos = df_hurtos[df_hurtos['Edad'] < 110]
# 3.Remoción de registros que tienen más de 20 valores iguales a NA, estos corresponden a las últimas filas que se leen
df_hurtos.dropna(thresh=10, inplace=True)

In [147]:
report = ProfileReport(df_hurtos)
report.to_file('Perfilamiento de hurto a vehículos.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 334.13it/s]


In [144]:
df_hurtos.describe()

,Edad,Cantidad
count,36546.000000,36546.000000
mean,40.620424,1.001067
std,12.642416,0.057054
min,0.000000,1.000000
25%,31.000000,1.000000
50%,38.000000,1.000000
75%,49.000000,1.000000
max,99.000000,9.000000


En promedio, el hurto afecta únicamente un vehículo a la vez. Las personas más afectadas corresponden a aquellas con edad promedio de 40 años

In [164]:
df_hurtos['Hora Militar'] = df_hurtos['Hora'].apply(lambda x: x[:2]).astype(int)

In [165]:
df_hurtos['Hora Militar'].value_counts()

0     3300
21    2471
20    2452
22    2164
19    2117
10    1778
23    1695
18    1549
14    1497
11    1474
15    1423
16    1359
17    1330
8     1313
9     1274
13    1244
12    1227
6     1184
1     1077
7     1073
5      933
2      932
3      903
4      777
Name: Hora Militar, dtype: int64